# 2. Self-regulation in disease spread

**Note:** $S, I$ and $R$ denote population _fractions_

**Assumptions:**
1. No seasonal forcing ($a = 0$).
1. Use the second-order Erlang kernel for the perceived hazard.
1. $I(t) = I_{0}$ for $t \leq 0$.

In [1]:
using DifferentialEquations
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())
using Integrals
using Plots
using Roots


#######################
# Mitigation function #
#######################

mitigation(h, h_thres, m_max, ϵ = 1/4000) = m_max - (m_max/h_thres)*ϵ*log( ( 1 + exp( (h_thres - h)/ϵ ) ) )

mitigation (generic function with 2 methods)

## a) As an integro-differential equation

The system is given by

\begin{equation}
    \begin{aligned}
        \dot{S} & = -\beta_{0} (1 - m(h)) S I + \nu R, \\ 
        \dot{I} & = \beta_{0} (1 - m(h)) S I - \gamma I, \\ 
        \dot{R} & = \gamma I - \nu R,
    \end{aligned}
\end{equation}

where the mitigation $m(h)$ and perceived hazard $h(t)$ are given by

\begin{equation}
    m(h) = m_{\text{max}} - \frac{m_{\text{max}}}{h_{\text{thres}}} \epsilon \log{\left\{1 + \exp{\left[\frac{1}{\epsilon}(h_{\text{thres}} - h)\right]} \right\}}, \qquad h(t) = I_{0} + \left( \frac{1}{\tau} \right)^{2} \mathrm{e}^{-t/\tau} \left[\int\limits_{0}^{t} I(t') \, (t - t') \mathrm{e}^{t'/\tau} \, \mathrm{d}t' \right]
\end{equation}


\begin{align*}
         h(t_{b}) = I_{0} + I(t_{a}) + \dot{I}(t_{a}) \left( \Delta t - 2 \tau \right) + \mathrm{e}^{-\Delta t / \tau} \left[ h(t_{a}) - I_{0} - \left( 1 + \frac{\Delta t}{\tau}\right) I(t_{a}) + \dot{I}(t_{a}) (\Delta t + 2 \tau) \right]
    \end{align*}

In [ ]:
##############
# Parameters #
##############

β0 = 0.5  # Basic spreading rate
γ = 0.1   # Recovery rate
ν = 0.01  # Rate of waning immunity
τ = 25    # Mitigation delay
h_thres = 1e-3  # Threshold hazard
m_max = 0.7  # Maximal mitigation

##################
# Initial values #
##################

s0 = 1 - h_thres
i0 = h_thres
r0 = 0.
h0 = h_thres

#######################
# Solution parameters #
#######################

# tf = 2.5*365.  # Final time in days.
tf = 10

# Save solutions at every...
Δt = 0.1 

h_int(i, di, h) = i0 + i + di*(Δt - 2*τ) + exp(-Δt/τ) * (h - i0 - (1 + (Δt/τ))*i + di*(Δt + 2*τ) )

function int_sirsm_model!(du, u, params, t)
    
    s, i, r, h = u
    
    β0, γ, ν, τ, h_thres, m_max = params
    
    m = mitigation(h, h_thres, m_max)
    
    du[1] = ds = -β0*(1 - m)*s*i + ν*r
    du[2] = di = β0*(1 - m)*s*i - γ*i
    du[3] = dr = γ*i - ν*r
    
    h_new = h_int(i, di, h)
    
    du[4] = (h_new - h)
end

###########
# Execute #
###########

u0 = [s0, i0, r0, h0]
params = [β0, γ, ν, τ, h_thres, m_max]
tspan = (0., tf)

int_sirsm_prob = ODEProblem(int_sirsm_model!, u0, tspan, params)
int_sirsm_sol = solve(int_sirsm_prob, saveat = Δt, reltol = 1e-10, abstol = 1e-10, progress=true);

## b) As a system of ODEs

In this case the system reads

\begin{equation}
    \begin{aligned}
        \dot{S} & = -\beta_{0} (1 - m(h)) S I + \nu R, \\ 
        \dot{I} & = \beta_{0} (1 - m(h)) S I - \gamma I, \\ 
        \dot{R} & = \gamma I - \nu R,
    \end{aligned} \qquad
    \begin{aligned}
        \dot{h} = \frac{1}{\tau}(h' - h), \\
        \dot{h}' = \frac{1}{\tau}(I - h').
    \end{aligned}
\end{equation}

With initial conditions related by $h_{0} = h'_{0} = I_{0}$

In [ ]:
function sirsm_model!(du, u, params, t)
    
    s, i, r, h, h2 = u
    
    β0, γ, ν, τ, h_thres, m_max = params
    
    m = mitigation(h, h_thres, m_max)
    
    du[1] = ds = -β0*(1 - m)*s*i + ν*r
    du[2] = di = β0*(1 - m)*s*i - γ*i
    du[3] = dr = γ*i - ν*r
    du[4] = dh = (h2 - h)/τ
    du[5] = dh2 = (i - h2)/τ
end

In [ ]:
##############
# Parameters #
##############

β0 = 0.5  # Basic spreading rate
γ = 0.1   # Recovery rate
ν = 0.01  # Rate of waning immunity
τ = 25    # Mitigation delay
h_thres = 1e-3  # Threshold hazard
m_max = 0.7  # Maximal mitigation

##################
# Initial values #
##################

s0 = 1 - h_thres
i0 = h_thres
r0 = 0.
h0 = h_thres
h20 = h_thres

#######################
# Solution parameters #
#######################

tf = 2.3*365.  # Final time in days.

# Save solutions at every...
Δt = 0.1 

###########
# Execute #
###########

u0 = [s0, i0, r0, h0, h20]
params = [β0, γ, ν, τ, h_thres, m_max]
tspan = (0., tf)

sirsm_prob = ODEProblem(sirsm_model!, u0, tspan, params)
sirsm_sol = solve(sirsm_prob, saveat = Δt, reltol = 1e-10, abstol = 1e-10, progress=true);
# sirsm_sol = solve(sirsm_prob, reltol = 1e-10, abstol = 1e-10, progress=true);

In [ ]:
# General plot attributes
kw =(;
    xlabel="Days",
    ylabel="Population fraction",
    xticks=0:100:tf,
    yticks=0:0.1:1,
    xrange=(-20, tf+10),
    yrange=(-0.04, 1.04),
    xminorgrid=true,
    xminorticks=4,
    yminorgrid=true,
    yminorticks=2,
    xlabelfontsize=11,
    ylabelfontsize=11,  # Default value
    legendfontsize=12,
    tickfontsize=9,
    framestyle=:box,
    legendposition=:topright,
    dpi=600)

labels = ["\$ S \$" "\$ I \$" "\$ R \$" "\$ h \$" "\$ h' \$"]
colors = [1 :red :green4 :blue :orange]
styles = [:solid :solid :solid :dash :dash]

plot(sirsm_sol, idxs = (0, 1:3), label=labels, color=colors, ls=styles, lw=1.4, legend_columns=3; kw...)
# plot!([tf/2, tf], [2/3, 2/3], lw=1.3, ls=:dash, color=:black, label=" ")
plot!([0.55*tf, tf], [0.303, 0.303], lw=1.5, ls=:dash, color=:black, label="\$ R^{*} \$")

## c) Fixed points and $\tau_{\text{crit}}$

In [11]:
decimal_places = 10
precision = 1/10^(decimal_places)

h_thres = 1e-3  # Threshold hazard
m_max = 0.7  # Maximal mitigation
ϵ = 1/4000 # Feedback curvature

β0 = 0.5  # Basic spreading rate
γ = 0.1   # Recovery rate
ν = 0.01  # Rate of waning immunity

m0 = mitigation(0, h_thres, m_max, ϵ)
upper_bound = ceil((ν / (ν + γ)) * (1 - γ/(β0*(1 - m0))), digits=decimal_places)

m1 = mitigation(1, h_thres, m_max, ϵ)
lower_bound = floor((ν / (ν + γ)) * (1 - γ/(β0*(1 - m1))), digits=decimal_places)

f(x) = x - (ν/(ν + γ))* (1 - γ/(β0*(1 - mitigation(x, h_thres, m_max, ϵ))))

i_fixed = find_zero(f, (lower_bound, upper_bound))

0.030303030303030307

In [ ]:
decimal_places = 10
precision = 1/10^(decimal_places)


h_thres = 1e-3  # Threshold hazard
m_max = 0.7  # Maximal mitigation
ϵ = 1/40 # Feedback curvature

β0 = 0.5  # Basic spreading rate
γ = 0.1   # Recovery rate
ν = 0.01  # Rate of waning immunity

m0 = mitigation(0, h_thres, m_max, ϵ)
upper_bound = ceil((ν / (ν + γ)) * (1 - γ/(β0*(1 - m0))), digits=decimal_places)

m1 = mitigation(1, h_thres, m_max, ϵ)
lower_bound = floor((ν / (ν + γ)) * (1 - γ/(β0*(1 - m1))), digits=decimal_places)

fixed_i(i, β0, γ, ν) = 1 - (γ*ν)/(β0*(ν - (ν + γ)*i))

i_range = lower_bound:precision:upper_bound

i_star = i_range[findmin(abs.(mitigation.(i_range, h_thres, m_max, ϵ) - fixed_i.(i_range, β0, γ, ν)))[2]]

println(i_star)

println("Lower bound: $(lower_bound)")
println("Upper bound: $(upper_bound)")

if lower_bound <= i_star <= upper_bound
    println("Bounds $(lower_bound) and $(upper_bound) are correct")
end

In [ ]:
# (ν / (ν + γ)) * (1 - γ/(β0*(1 - m1)))
@. i_range - (ν/(ν + γ))* (1 - γ/(β0*(1 - mitigation(i_range, h_thres, m_max, ϵ))))